In [1]:
import numpy as np
import scipy.stats as stats
import vectorbt as vbt

In [3]:
# load market price data
start = "2016-01-01 UTC"
end = "2020-01-01 UTC"
prices = vbt.YFData.download(
    "AAPL",
    start=start,
    end=end
).get("Close")

In [29]:
# split data for walk-forward optimization
# 30 splits, each two years long and reserves 180 days for the test
(in_price, in_indexes), (out_price, out_indexses) = prices.vbt.rolling_split(
    n=30,
    window_len=365 * 2,
    set_lens=(180,),
    left_to_right=False,
)

In [7]:
# function that returns the sharpe ratios for all combinations of moving average windows
def simulate_all_params(price, windows, **kwargs):
    fast_ma, slow_ma = vbt.MA.run_combs(
        price,
        windows,
        r=2,
        short_names=["fast", "slow"]
    )
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    pf = vbt.Portfolio.from_signals(price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [13]:
# helper functions that return the indexes and parameters where the performance is maximized
def get_best_index(performance):
    return performance[
        performance.groupby("split_idx").idxmax()
    ].index
def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [21]:
# A function that runs the backtest given the best moving average values and returns the associated Sharpe ratio
def simulate_best_params(price, best_fast_windows, best_slow_windows, **kwargs):
    fast_ma = vbt.MA.run(
        price,
        window=best_fast_windows,
        per_column=True
    )
    slow_ma = vbt.MA.run(
        price,
        window=best_slow_windows,
        per_column=True
    )
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    pf = vbt.Portfolio.from_signals(
        price, entries, exits, **kwargs)
    return pf.sharpe_ratio()

In [23]:
# execute the analysis by passing in a range of moving average windows to simulute_all_params
# returns the sharpe ratio for every combination
windows = np.arange(10, 40)
in_sharpe = simulate_all_params(
    in_price,
    windows,
    direction="both",
    freq="d"
)

In [24]:
# return the best in-sample moving average windows and combine them into one array
in_best_index = get_best_index(in_sharpe)
in_best_fast_windows = get_best_params(
    in_best_index,
    "fast_window"
)
in_best_slow_windows = get_best_params(
    in_best_index,
    "slow_window"
)
in_best_window_pairs = np.array(
    list(
        zip(
            in_best_fast_windows,
            in_best_slow_windows
        )
    )
)

In [37]:
#retrieve the out-of-sample Sharpe ratios using the optimized moving average windows
out_test_sharpe = simulate_best_params(
    out_price,
    in_best_fast_windows,
    in_best_slow_windows,
    direction="both",
    freq="d"
)

In [33]:
display(out_test_sharpe)

ma_window  ma_window  split_idx
10         11         0            0.104944
12         13         1            0.318348
                      2            0.971242
10         11         3            1.386778
12         13         4            1.303290
10         11         5            2.133301
                      6            2.043528
18         23         7            1.756913
                      8            2.219375
                      9            2.283887
                      10           2.544001
                      11           2.724653
                      12           2.389905
                      13           2.838682
                      14           2.393308
                      15           1.116318
23         26         16           0.670548
                      17           0.594139
                      18           0.816456
24         25         19           0.276340
                      20          -0.052502
                      21          -0.363481
